In [2]:
import requests
from bs4 import BeautifulSoup
import re

response = requests.get("https://www.handbook.uts.edu.au/subjects/alpha.html")
soup = BeautifulSoup(response.text, "html.parser")

In [10]:
links = soup.find_all('a', href=True)
pattern = re.compile(r'/\d{5}\.html')
all_subject_links = [x["href"] for x in links if pattern.search(x["href"])]
print(all_subject_links[:10])


['https://handbook.uts.edu.au/subjects/32998.html', 'https://handbook.uts.edu.au/subjects/32543.html', 'https://handbook.uts.edu.au/subjects/42145.html', 'https://handbook.uts.edu.au/subjects/42890.html', 'https://handbook.uts.edu.au/subjects/92681.html', 'https://handbook.uts.edu.au/subjects/92360.html', 'https://handbook.uts.edu.au/subjects/92376.html', 'https://handbook.uts.edu.au/subjects/92389.html', 'https://handbook.uts.edu.au/subjects/59720.html', 'https://handbook.uts.edu.au/subjects/59721.html']


In [22]:
from dataclasses import dataclass
from enum import Enum, auto

class SubjectType(Enum):
    Undergraduate = auto()
    Postgraduate = auto()

@dataclass
class SubjectInfo:
    subject_type: SubjectType
    number: int
    name: str
    prereqs: list[int]
    area: str

In [27]:
print(all_subject_links[3])

https://handbook.uts.edu.au/subjects/42890.html


In [26]:

all_subjects = []

for subject_link in all_subject_links[:5]:
    subject_number = subject_link.split("/")[-1][:-5]

    response = requests.get(subject_link)
    if response.status_code != 200:
        print(f"Unsuccessful: {response.status_code}")
        print(response)
        break

    soup = BeautifulSoup(response.text, "html.parser")


    course_area = soup.find("a", class_="coursearea").text

    requisite_el = [x for x in soup.find_all("em") if x.text.strip().startswith("Requisite")]
    requisites = [int(x.text) for x in requisite_el[0].find_all("a")] if requisite_el else []

    subject_name = soup.find("h1").text

    subject_type_str = [x.text for x in soup.find_all("p") if x.text == "Undergraduate" or x.text == "Postgraduate"][0]
    subject_type = SubjectType[subject_type_str]
    
    new_subj = SubjectInfo(subject_type, subject_number, subject_name, requisites, course_area)
    all_subjects.append(new_subj)
    print(new_subj)


SubjectInfo(subject_type=<SubjectType.Postgraduate: 2>, number='32998', name='32998 .NET Application Development', prereqs=[32555], area='Information Technology')
SubjectInfo(subject_type=<SubjectType.Postgraduate: 2>, number='32543', name='32543 3D Animation', prereqs=[], area='Information Technology')


IndexError: list index out of range